In [1]:

import pandas as pd
import numpy as np

FLIGHT_FILE   = "Flight Level Data.csv"
BAG_FILE      = "Bag+Level+Data.csv"
PNR_FILE      = "PNR+Flight+Level+Data.csv"
REMARKS_FILE  = "PNR Remark Level Data.csv"
OUTPUT_FILE   = "Flight_Difficulty_Score_final_with_rank_and_classification.csv"


 
def safe_minmax_ser(s):
    s2 = s.dropna()
    if len(s2) == 0:
        return 0.0, 1.0
    return s2.min(), s2.max()

def minmax_scale(series, lo=None, hi=None):
    s = series.copy().astype(float)
    if lo is None or hi is None:
        lo, hi = safe_minmax_ser(s)
    if hi == lo:
        return pd.Series(0.0, index=s.index)
    return (s - lo) / (hi - lo)

def drop_duplicated_columns(df):
    if df.columns.duplicated().any():
        df = df.loc[:, ~df.columns.duplicated()]
    return df

def detect_date_col(df):
    for c in df.columns:
        low = c.lower()
        if 'scheduled_departure' in low or 'flight_date' in low or low=='date':
            return c
    # fallback to any date-like column name
    for c in df.columns:
        if 'date' in c.lower():
            return c
    return None

# Loading flight-level data and using basic features 
fl = pd.read_csv(FLIGHT_FILE)

# Getting scheduled and actual departure columns
sched_col = next((c for c in fl.columns if 'scheduled_departure' in c.lower() and 'datetime' in c.lower()), 
                 next((c for c in fl.columns if 'scheduled_departure' in c.lower()), None))
actual_col = next((c for c in fl.columns if 'actual_departure' in c.lower() and 'datetime' in c.lower()), 
                  next((c for c in fl.columns if 'actual_departure' in c.lower()), None))

if sched_col is None:
    raise KeyError("Could not find scheduled departure datetime column in FlightLevelData.csv")

fl[sched_col] = pd.to_datetime(fl[sched_col], errors='coerce')
if actual_col:
    fl[actual_col] = pd.to_datetime(fl[actual_col], errors='coerce')

# flight date
fl['flight_date'] = fl[sched_col].dt.date

# departure delay in minutes
if actual_col:
    fl['departure_delay_min'] = (fl[actual_col] - fl[sched_col]).dt.total_seconds() / 60.0
else:
    delay_col = next((c for c in fl.columns if 'dep_delay' in c.lower() or 'departure_delay' in c.lower()), None)
    fl['departure_delay_min'] = pd.to_numeric(fl[delay_col], errors='coerce') if delay_col else np.nan

# numeric ground times
fl['scheduled_ground_time_minutes'] = pd.to_numeric(fl.get('scheduled_ground_time_minutes'), errors='coerce')
fl['minimum_turn_minutes'] = pd.to_numeric(fl.get('minimum_turn_minutes'), errors='coerce')

# ground pressure and turnaround slack
fl['ground_pressure'] = np.where((fl['scheduled_ground_time_minutes'].notna()) & (fl['scheduled_ground_time_minutes']>0),
                                fl['minimum_turn_minutes'] / fl['scheduled_ground_time_minutes'],
                                np.nan)
fl['turnaround_slack'] = fl['scheduled_ground_time_minutes'] - fl['minimum_turn_minutes']

# day of week, departure hour
fl['day_of_week'] = fl[sched_col].dt.dayofweek
fl['departure_hour'] = fl[sched_col].dt.hour

# haul type derived from scheduled_ground_time_minutes
def haul_type_from_ground(x):
    if pd.isna(x): return np.nan
    if x < 60: return 1
    if x <= 120: return 2
    return 3
fl['haul_type'] = fl['scheduled_ground_time_minutes'].apply(haul_type_from_ground)

# standardizing destination airport column name (using scheduled_arrival_station_code if present)
dest_col = next((c for c in fl.columns if 'arrival' in c.lower() and ('station' in c.lower() or 'airport' in c.lower() or 'destination' in c.lower() or 'dest' in c.lower())), None)
if dest_col:
    fl = fl.rename(columns={dest_col:'destination_airport_code'})
else:
    fl['destination_airport_code'] = np.nan

flight_core = fl[['company_id','flight_number','flight_date','departure_delay_min',
                  'scheduled_ground_time_minutes','minimum_turn_minutes','ground_pressure',
                  'turnaround_slack','day_of_week','departure_hour','haul_type','destination_airport_code']].copy()
flight_core = drop_duplicated_columns(flight_core)

# Bag-level features: transfer, hot-transfer, checked 
bags = pd.read_csv(BAG_FILE)
bags['bag_type'] = bags['bag_type'].astype(str).str.strip().str.title()

# Column in bag file if present
bag_date_col = detect_date_col(bags)
if bag_date_col:
    bags[bag_date_col] = pd.to_datetime(bags[bag_date_col], errors='coerce').dt.date
    bags = bags.rename(columns={bag_date_col:'flight_date'})
else:
    bags['flight_date'] = np.nan

bags['is_transfer'] = bags['bag_type'].str.contains('Transfer', case=False, na=False).astype(int)
bags['is_hot_transfer'] = bags['bag_type'].str.contains('Hot Transfer', case=False, na=False).astype(int)
bags['is_checked'] = bags['bag_type'].str.contains('Origin|Checked', case=False, na=False).astype(int)

bag_group_keys = ['flight_number','flight_date']
if 'company_id' in bags.columns:
    bag_group_keys.insert(0,'company_id')

bag_stats = bags.groupby(bag_group_keys).agg(
    transfer_bags=('is_transfer','sum'),
    hot_transfer_bags=('is_hot_transfer','sum'),
    checked_bags=('is_checked','sum')
).reset_index()
bag_stats['total_bags'] = bag_stats['transfer_bags'] + bag_stats['checked_bags']

def safe_ratio(n,d):
    if d > 0:
        return n/d
    elif d == 0 and n > 0:
        return 5.0  # capped penalty for transfer-only flights
    else:
        return 0.0

bag_stats['transfer_to_checked_ratio'] = bag_stats.apply(lambda r: safe_ratio(r['transfer_bags'], r['checked_bags']), axis=1)
bag_stats['hot_transfer_ratio'] = bag_stats.apply(lambda r: safe_ratio(r['hot_transfer_bags'], r['checked_bags']), axis=1)
bag_stats = drop_duplicated_columns(bag_stats)

# Passenger loads 
pnr = pd.read_csv(PNR_FILE)
pnr_date_col = detect_date_col(pnr)
if pnr_date_col is None:
    raise KeyError("Could not find a date-like column in PNR file.")
pnr[pnr_date_col] = pd.to_datetime(pnr[pnr_date_col], errors='coerce').dt.date
pnr = pnr.rename(columns={pnr_date_col:'flight_date'})

if 'total_pax' not in pnr.columns:
    alt = next((c for c in pnr.columns if 'pax' in c.lower() or 'pass' in c.lower()), None)
    if alt is None:
        raise KeyError("PNR file missing 'total_pax' column.")
    pnr = pnr.rename(columns={alt:'total_pax'})

pnr['total_pax'] = pd.to_numeric(pnr['total_pax'], errors='coerce').fillna(0)
pax_group_keys = ['flight_number','flight_date']
if 'company_id' in pnr.columns:
    pax_group_keys.insert(0,'company_id')
pax_loads = pnr.groupby(pax_group_keys)['total_pax'].sum().reset_index().rename(columns={'total_pax':'pax_count'})
pax_loads = drop_duplicated_columns(pax_loads)

# SSR counts 
remarks = pd.read_csv(REMARKS_FILE)
text_col = 'special_service_request' if 'special_service_request' in remarks.columns else next((c for c in remarks.columns if any(k in c.lower() for k in ['remark','text','note','comment'])), None)
if text_col is None:
    raise KeyError("No remark/text column found in remarks file.")
remarks[text_col] = remarks[text_col].astype(str).str.lower()

ssr_keywords = ['wheel','airport wheelchair','manual wheelchair','unaccompanied minor','unaccompanied','umnr','stroller','medical','assist']
remarks['is_ssr'] = remarks[text_col].apply(lambda s: int(any(k in s for k in ssr_keywords)))

rem_date_col = detect_date_col(remarks)
if rem_date_col:
    remarks[rem_date_col] = pd.to_datetime(remarks[rem_date_col], errors='coerce').dt.date
    remarks = remarks.rename(columns={rem_date_col:'flight_date'})
else:
    remarks['flight_date'] = np.nan

ssr_keys = ['flight_number','flight_date']
if 'company_id' in remarks.columns:
    ssr_keys.insert(0,'company_id')
ssr_counts = remarks.groupby(ssr_keys)['is_ssr'].sum().reset_index().rename(columns={'is_ssr':'ssr_count'})
ssr_counts = drop_duplicated_columns(ssr_counts)

#  Destination congestion index (avg departure delay by destination) 
if 'destination_airport_code' in fl.columns:
    dc = fl.groupby('destination_airport_code')['departure_delay_min'].mean().reset_index().rename(columns={'departure_delay_min':'mean_delay'})
    dc['dest_congestion'] = minmax_scale(dc['mean_delay'])
    dest_congestion = dc[['destination_airport_code','dest_congestion']].copy()
else:
    dest_congestion = pd.DataFrame(columns=['destination_airport_code','dest_congestion'])

dest_congestion = drop_duplicated_columns(dest_congestion)

# ---------- 6) Merge sources into master ----------
master = pax_loads.copy()

merge_b_keys = ['company_id','flight_number','flight_date'] if ('company_id' in bag_stats.columns and 'company_id' in master.columns) else ['flight_number','flight_date']
master = master.merge(bag_stats, how='left', on=merge_b_keys)
master = drop_duplicated_columns(master)

merge_f_keys = ['company_id','flight_number','flight_date'] if ('company_id' in flight_core.columns and 'company_id' in master.columns) else ['flight_number','flight_date']
master = master.merge(flight_core, how='left', on=merge_f_keys)
master = drop_duplicated_columns(master)

merge_s_keys = ['company_id','flight_number','flight_date'] if ('company_id' in ssr_counts.columns and 'company_id' in master.columns) else ['flight_number','flight_date']
master = master.merge(ssr_counts, how='left', on=merge_s_keys)
master = drop_duplicated_columns(master)

if not dest_congestion.empty and 'destination_airport_code' in master.columns:
    master = master.merge(dest_congestion, how='left', on='destination_airport_code')
else:
    master['dest_congestion'] = 0.0

master = drop_duplicated_columns(master)

# ---------- 7) Clean/fill numeric columns and derive final features ----------
for c in ['transfer_bags','hot_transfer_bags','checked_bags','total_bags','transfer_to_checked_ratio','hot_transfer_ratio']:
    if c in master.columns:
        master[c] = pd.to_numeric(master[c], errors='coerce').fillna(0)

master['pax_count'] = pd.to_numeric(master.get('pax_count', 0)).fillna(0)
master['departure_delay_min'] = pd.to_numeric(master.get('departure_delay_min', 0)).fillna(0)
master['ground_pressure'] = pd.to_numeric(master.get('ground_pressure', np.nan))
master['turnaround_slack'] = pd.to_numeric(master.get('turnaround_slack', 0)).fillna(0)
master['ssr_count'] = pd.to_numeric(master.get('ssr_count', 0)).fillna(0)
master['day_of_week'] = pd.to_numeric(master.get('day_of_week', master['flight_date'].apply(lambda d: pd.to_datetime(d).dayofweek if pd.notna(d) else 0))).fillna(0)
master['haul_type'] = pd.to_numeric(master.get('haul_type', 1)).fillna(1)
master['dest_congestion'] = pd.to_numeric(master.get('dest_congestion', 0)).fillna(0)

master['total_bags'] = master.get('total_bags', 0).fillna(0)
master['bags_per_pax'] = np.where(master['pax_count']>0, master['total_bags'] / master['pax_count'], 0.0)
master['ssr_rate'] = np.where(master['pax_count']>0, (master['ssr_count'] / master['pax_count']) * 100.0, 0.0)
master['transfer_to_checked_ratio'] = master.get('transfer_to_checked_ratio', 0).fillna(0)
master['hot_transfer_ratio'] = master.get('hot_transfer_ratio', 0).fillna(0)

# Feature mapping and weights (11 variables/metrics) 
feat_map = {
    'departure_delay_min': ('delay_norm', 1.0),
    'ground_pressure': ('ground_norm', 1.0),
    'turnaround_slack': ('slack_norm', 1.0),
    'pax_count': ('pax_norm', 1.0),
    'transfer_to_checked_ratio': ('bagratio_norm', 1.0),
    'hot_transfer_ratio': ('hotratio_norm', 1.5),
    'bags_per_pax': ('bagsperpax_norm', 0.5),
    'ssr_rate': ('ssr_norm', 1.0),
    'day_of_week': ('dow_norm', 0.5),
    'haul_type': ('haul_norm', 0.5),
    'dest_congestion': ('dest_norm', 0.75)
}

# Normalizing (0-1). invert slack (lower slack -> harder) 
for col, (outcol, weight) in feat_map.items():
    if col not in master.columns:
        master[col] = 0.0
    lo, hi = safe_minmax_ser(master[col])
    scaled = minmax_scale(master[col], lo, hi).fillna(0.0)
    if col == 'turnaround_slack':
        scaled = 1.0 - scaled
    master[outcol] = scaled

# Composite weighted sum 
master['raw_fds'] = 0.0
for col, (outcol, weight) in feat_map.items():
    master['raw_fds'] += master[outcol] * weight

master['fds'] = minmax_scale(master['raw_fds'])

# Daily ranking (highest FDS = most difficult) & classification into Easy/Medium/Difficult
def classify_daily(df):
    df = df.copy()
    if df['fds'].nunique() == 1:
        df['fds_category'] = 'Medium'
        df['fds_rank'] = 1
        return df
    q1 = df['fds'].quantile(0.25)
    q3 = df['fds'].quantile(0.75)
    df['fds_category'] = df['fds'].apply(lambda x: 'Difficult' if x >= q3 else ('Easy' if x <= q1 else 'Medium'))
    df['fds_rank'] = df['fds'].rank(ascending=False, method='dense').astype(int)
    return df

master = master.groupby('flight_date', group_keys=False).apply(classify_daily).reset_index(drop=True)

# Save and preview 
output_cols = [
    'company_id','flight_number','flight_date','pax_count',
    'departure_delay_min','ground_pressure','turnaround_slack','transfer_to_checked_ratio','hot_transfer_ratio',
    'bags_per_pax','ssr_rate','day_of_week','haul_type','dest_congestion',
    'raw_fds','fds','fds_rank','fds_category'
]
out = master[[c for c in output_cols if c in master.columns]].copy()
out.to_csv(OUTPUT_FILE, index=False)
print("Saved final FDS with daily rank & category to:", OUTPUT_FILE)

# sample top difficult flights
print("\nTop 10 (by FDS) sample:")
print(out.sort_values(['flight_date','fds'], ascending=[True, False]).groupby('flight_date').head(1).head(10).to_string(index=False))


<ipython-input-1-53ab13a6c9cb>:282: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  master = master.groupby('flight_date', group_keys=False).apply(classify_daily).reset_index(drop=True)


Saved final FDS with daily rank & category to: Flight_Difficulty_Score_final_with_rank_and_classification.csv

Top 10 (by FDS) sample:
company_id  flight_number flight_date  pax_count  departure_delay_min  ground_pressure  turnaround_slack  transfer_to_checked_ratio  hot_transfer_ratio  bags_per_pax  ssr_rate  day_of_week  haul_type  dest_congestion  raw_fds      fds  fds_rank fds_category
        UA            944  2025-08-01        399                 90.0         0.620000              95.0                        0.0                 0.0      0.000000  0.000000          4.0        3.0         0.398675 2.965966 0.568285         1    Difficult
        UA           1599  2025-08-02        530                  6.0         0.500000              93.0                        0.0                 0.0      0.000000  0.000000          5.0        3.0         0.160901 3.035491 0.588250         1    Difficult
        UA            845  2025-08-03        370                  7.0         0.775000     